<a href="https://colab.research.google.com/github/tlsdmswn01/Text_mining_project/blob/main/%EA%B8%B0%EC%97%85%20%EC%A0%95%EB%B3%B4%20%EC%A0%9C%EA%B3%B5-%EA%B5%B0%EC%A7%91%ED%99%94%2C%20%EC%9C%A0%EC%82%AC%EB%8F%84%20%ED%99%9C%EC%9A%A9/%EA%B8%B0%EC%82%AC_%EA%B5%B0%EC%A7%91%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 뉴스[제목]으로 비슷한 뉴스 군집화

In [ ]:
import pandas as pd

df = pd.read_excel('삼성전자_주가_2020_2023.xlsx')

df['일자'] = df['일자'].astype('int')
df = df[df['일자'] >= 20230501] # 2023년 5월 이후의 데이터만 사용(최근 뉴스 제공 위해)

df = df[['일자', '언론사', '제목', '본문', '키워드' ,'URL']]

# 제목에서 [], () 안에 있는 내용 지우기
df['제목'] = df['제목'].apply(lambda x : re.sub(r'\[[^]]*\]', '', x))
df['제목'] = df['제목'].apply(lambda x : re.sub(r'\([^)]*\)', '', x))
df

In [ ]:
!pip install konlpy

from konlpy.tag import Okt

okt = Okt() # 형태소 분석기 객체 생성
noun_list = []
for content in df['제목']:
    nouns = okt.nouns(content) # 명사만 추출하기, 결과값은 명사 리스트
    noun_list.append(nouns)

df['nouns'] = noun_list  # 제목에서 추출한 명사

drop_index_list = [] # 지워버릴 index를 담는 리스트
for i, row in df.iterrows():
    temp_nouns = row['nouns']
    if len(temp_nouns) == 0: # 만약 명사리스트가 비어 있다면
        drop_index_list.append(i) # 지울 index 추가

df = df.drop(drop_index_list) # 해당 index를 지우기

# index를 지우면 순회시 index 값이 중간중간 비기 때문에 index를 다시 지정
df.index = range(len(df))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 문서를 명사 집합으로 보고 문서 리스트로 치환 (tfidfVectorizer 인풋 형태를 맞추기 위해)
text = [" ".join(noun) for noun in df['nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()
vector = np.array(vector) # Normalizer를 이용해 변환된 벡터

In [ ]:
# DBSCAN Clustering
from sklearn.cluster import DBSCAN
import numpy as np

model = DBSCAN(eps=0.3,min_samples=5, metric = "cosine")
# 거리 계산 식으로는 Cosine distance를 이용
result = model.fit_predict(vector)
df['cluster1st'] = result

print('군집개수 :', result.max())
df

In [ ]:
# TF-IDF 기준으로 각 cluster group 중 하나의 기사를 뽑고
# 각 그룹에서 빈도 높은 키워드 뽑음

clusters = []
counts = []
top_title = []
top_noun = []
for cluster_num in set(result):
    # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
    # if(cluster_num == -1 or cluster_num == 0):
    #     continue
    # else:
        print("cluster num : {}".format(cluster_num))
        temp_df = df[df['cluster1st'] == cluster_num] # cluster num 별로 조회
        clusters.append(cluster_num)
        counts.append(len(temp_df))
        top_title.append(temp_df.reset_index()['제목'][0])
        top_noun.append(temp_df.reset_index()['nouns'][0]) # 군집별 첫번째 기사를 대표기사로 ; tfidf방식
        for title in temp_df['제목']:
            print(title) # 제목으로 살펴보자
        print()

cluster_result = pd.DataFrame({'cluster_num':clusters, 'count':counts, 'top_title':top_title, 'top_noun':top_noun})
cluster_result

### 본문[키워드]로 군집별 키워드 뽑기

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT

def BERT(title):
    # 1번째 군집
    array_text = pd.DataFrame(df[df['cluster1st'] == 1]['키워드']).to_numpy()

    bow = []
    from keybert import KeyBERT
    kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
    for j in range(len(array_text)):
        keywords = kw_extractor.extract_keywords(array_text[j][0])
        bow.append(keywords)

    new_bow = []
    for i in range(0, len(bow)):
        for j in range(len(bow[i])):
            new_bow.append(bow[i][j])

    keyword = pd.DataFrame(new_bow, columns=['keyword', 'weight'])
    print(keyword.groupby('keyword').agg('sum').sort_values('weight', ascending=False).head(20))

In [ ]:
BERT('삼성전자')  # '삼성전자'와 가장 관련이 높은 키워드